<a href="https://colab.research.google.com/github/TarunRao-K/jammingSDR/blob/main/LoRa_Classifier_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LoRa Signal Classifier

Imports and constants

In [ ]:
import os, re, random
from collections import defaultdict
import numpy as np
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from scipy.signal import spectrogram
import matplotlib.pyplot as plt

PATH_DATASETS = '/content/drive/MyDrive/LoRa_Detection'
PATH_MODELS = '/content/drive/MyDrive/LoRa_Detection/Models'
PATH_PREAMBLE_REFERENCE_IQ = '/content/drive/MyDrive/LoRa_Detection/Signal_Data/Preamble_symbols'
PATH_RESULTS = '/content/drive/MyDrive/LoRa_Detection/Code'

F_SAMP = int(1e6)
F_MAX = 250000
WINDOW_SIZE = 2**7
WINDOW_OVERLAP = WINDOW_SIZE // 2

BW_VALUES = [125000, 250000, 500000]
SF_VALUES = [7, 8, 9, 10, 11, 12]
NUM_CLASSES = 18

MODEL_INPUT_SHAPE = (1024, 1)
MODEL_OUTPUT_SHAPE = (NUM_CLASSES,)

SNR_MIN = -20
SNR_MAX = 20
SNR_STEP = 2

Utility Functions

In [ ]:
def encode_label(config):
  one_hot_label = np.zeros(NUM_CLASSES)
  one_hot_label[config] = 1
  return one_hot_label

def decode_snr(snr):
    # Decode SNR from file name (sign: 0/1, magnitude: 0.00-99.99)
    snr_sign, snr_mag = divmod(int(snr), 10000)
    snr = (-1)*(snr_mag/100) if snr_sign else snr_mag/100
    return snr

def parse_file_name(file_path):
    file_name = os.path.basename(file_path)
    match = re.search(r'SNR(\d{5})D(\d{2})C(\d{2})T(\d{2}).npy', file_name)
    if match:
        snr = decode_snr(match.group(1))
        dist = int(match.group(2))
        config = int(match.group(3))
        index = int(match.group(4))
        return snr, dist, config, index
    else:
        return None

def filter_files(dataset_name, filter_snr=None, filter_config=None, filter_count=50, shuffle=True):
    files_list = []
    path_dataset = os.path.join(PATH_DATASETS, dataset_name)
    all_files = os.listdir(path_dataset)
    for file_name in all_files:
        snr, _, config, index = parse_file_name(file_name)       
        if filter_snr is not None:
            if int(snr) not in filter_snr:
                continue
        if filter_config is not None:
            if config not in filter_config:
                continue
        if index > filter_count:
          continue
        files_list.append(os.path.join(path_dataset, file_name))
    if shuffle:
        np.random.shuffle(files_list)
    return files_list

def read_iq_file(filepath):
    with open(filepath, "rb") as f:
        iq_samples = np.fromfile(f, dtype=np.complex64)
    return iq_samples

def read_IF_sample(file_path):
  IF_sample = np.load(file_path)[-MODEL_INPUT_SHAPE[0]:]
  return IF_sample

def get_reference_iq_samples(bandwidth, spread_factor):
  iq_file_path = os.path.join(PATH_PREAMBLE_REFERENCE_IQ,
                                  'PR_Ref_BW{}_SF{}.bin'.format(
                                      BW_VALUES.index(bandwidth),
                                      spread_factor))
  return read_iq_file(iq_file_path)

def pad_signal(signal_IQ):
  signal_len = signal_IQ.shape[0]
  input_length= (MODEL_INPUT_SHAPE[0]+1)*WINDOW_OVERLAP

  padded_signal = np.zeros(input_length, dtype= np.complex64)
  if signal_len <= input_length:
    padded_signal[-signal_len:] = signal_IQ
  else:
    padded_signal = signal_IQ[-input_length:]
  return padded_signal

def add_gaussian_noise(signal, snr_db):
    non_zero_samples = np.nonzero(signal)[0]  
    first_non_zero = non_zero_samples[0] if len(non_zero_samples) > 0 else 0
    last_non_zero = non_zero_samples[-1] if non_zero_samples[-1] < len(
        signal) else len(signal)-1
    signal_power = np.mean(np.abs(signal[first_non_zero:last_non_zero+1]) ** 2)
    snr = 10 ** (snr_db / 10.0)
    noise_power = signal_power / snr
    noise = np.sqrt(noise_power / 2) * (np.random.randn(*signal.shape) + 1j * np.random.randn(*signal.shape))    
    noisy_signal = signal + noise
    return noisy_signal

def simulate_iq_samples(bandwidth, spread_factor, snr_db):
    IQ_ref = get_reference_iq_samples(bandwidth, spread_factor)
    IQ_noisy = add_gaussian_noise(pad_signal(IQ_ref), snr_db)
    return IQ_noisy

def compute_spectrogram(IQ_samples):
  Sxx =  spectrogram(IQ_samples, F_SAMP,window= 'hann',
                             nperseg= WINDOW_SIZE,noverlap= WINDOW_OVERLAP,
                             return_onesided= False)[2]
  Sxx = np.vstack([Sxx[Sxx.shape[0]//2:], Sxx[:Sxx.shape[0]//2]])
  Sxx = Sxx[WINDOW_SIZE//4:-WINDOW_SIZE//4, :]
  Sxx = (Sxx - np.min(Sxx))/(np.max(Sxx)-np.min(Sxx))
  return Sxx

def compute_inst_freq(spec_sample):
  spec_sample = spec_sample**4
  f_bins = np.linspace(-F_MAX, F_MAX, num=spec_sample.shape[0]+1)[:-1]
  weighted_sum = np.sum(spec_sample * f_bins[:, np.newaxis], axis=0)
  total_power = np.sum(spec_sample, axis=0)
  IF_sample = np.clip((weighted_sum / total_power)/F_MAX, -1, 1)[:, np.newaxis]
  return IF_sample

def simulate_IF_sample(config, snr):
  bandwidth, spread_factor = BW_VALUES[config//6], SF_VALUES[config%6]
  IQ_samples = simulate_iq_samples(bandwidth, spread_factor, snr)
  spec_sample = compute_spectrogram(IQ_samples)
  IF_sample = compute_inst_freq(spec_sample)[-MODEL_INPUT_SHAPE[0]:]
  return IF_sample

Dataset

In [ ]:
def data_generator(files_list, batch_size):
  while True:
    batch_data, batch_labels = [], []
    batch_sample_count = 0
    for file_path in files_list:
      sample = read_IF_sample(file_path)
      sample = sample * 2 if 'sim' not in file_path.lower() else sample
      config = parse_file_name(os.path.basename(file_path))[2]
      label = encode_label(config)
      if sample.shape==MODEL_INPUT_SHAPE and label.shape==MODEL_OUTPUT_SHAPE:
        batch_data.append(sample)
        batch_labels.append(label)
        batch_sample_count += 1
      if batch_sample_count >= batch_size:
        indices = np.arange(batch_sample_count)
        np.random.shuffle(indices)
        yield np.array(batch_data)[indices], np.array(batch_labels)[indices]
        batch_data, batch_labels = [], []
        batch_sample_count = 0

def simulation_data_generator(num_samples_per_case, snr_range, config_range, batch_size):
  while True:
    batch_data, batch_labels = [], []
    batch_sample_count = 0
    for _ in range(num_samples_per_case):
      for snr in snr_range:
        for config in config_range:
            sample = simulate_IF_sample(config, snr)
            label = encode_label(config)
            if sample.shape==MODEL_INPUT_SHAPE and label.shape==MODEL_OUTPUT_SHAPE:
              batch_data.append(sample)
              batch_labels.append(label)
              batch_sample_count += 1
            if batch_sample_count >= batch_size:
              indices = np.arange(batch_sample_count)
              np.random.shuffle(indices)
              yield np.array(batch_data)[indices], np.array(batch_labels)[indices]
              batch_data, batch_labels = [], []
              batch_sample_count = 0

Model

In [ ]:
def create_model():
  model = Sequential([
      Flatten(input_shape=MODEL_INPUT_SHAPE),
      Dense(16, activation='tanh'),
      Dense(16, activation='tanh'),
      Dropout(0.5),
      Dense(NUM_CLASSES, activation='softmax')
  ])
  model.compile(loss=tf.keras.losses.categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
  return model

def load_model(model_name):
  path_model = os.path.join(PATH_MODELS, model_name)
  return tf.keras.models.load_model(path_model)

def save_model(model, model_name):
  path_model = os.path.join(PATH_MODELS, model_name)
  model.save(path_model)

def train_model_from_files(model, train_data, epochs=5, batch_size=8, validation_data=None):
  num_samples = len(train_data)
  steps_per_epoch = num_samples // batch_size
  train_data_gen = data_generator(train_data, batch_size)
  if validation_data is not None:
    num_val_samples = len(validation_data)
    validation_steps = num_val_samples // batch_size
    val_data_gen = data_generator(validation_data, batch_size)
  else:
    num_samples_per_case = 50
    snr_range = range(SNR_MIN, SNR_MAX+1, SNR_STEP)
    config_range = range(NUM_CLASSES)
    num_val_samples = num_samples_per_case*len(snr_range)*len(config_range)
    validation_steps = num_val_samples // batch_size
    val_data_gen = simulation_data_generator(num_samples_per_case, snr_range, 
                                          config_range, batch_size)
  model.fit(train_data_gen, epochs=epochs, steps_per_epoch=steps_per_epoch, 
            validation_data=val_data_gen, validation_steps=validation_steps,
            verbose=1)
  return model

def model_evaluate_sim(model, snr_range = list(range(SNR_MIN, SNR_MAX+1, SNR_STEP)), config_range = list(range(NUM_CLASSES)), num_samples= 32):
  sim_results = {}
  for snr in snr_range:
    samples = np.concatenate([np.array([
        simulate_IF_sample(config, snr) for _ in range(num_samples)
    ]) for config in config_range])
    labels = np.concatenate([np.array(
        [encode_label(config)]*num_samples
        ) for config in config_range])
    predictions = model.predict(samples)
    sim_results[snr] = np.stack((predictions, labels), axis = 1)
  return sim_results

Simulation

In [ ]:
def MC_CV_sim(filter_snr = list(range(0, 20, 4)), num_samples= 50, 
              num_iterations=10, num_epochs= 30, val_split=0.2, batch_size=8):
  sim_files_list = filter_files('Sim_IF_Dataset2', filter_snr=filter_snr, filter_count=num_samples)
  results = []
  for i in range(num_iterations):
    print(f'MC-CV iteration {i+1}')
    random.shuffle(sim_files_list)
    split_idx = int(len(sim_files_list)*(1-val_split))
    train_files = sim_files_list[:split_idx]
    validation_files = sim_files_list[split_idx:]
    model_i = create_model()
    model_i = train_model_from_files(model_i, train_data= train_files, validation_data= validation_files, epochs=num_epochs, batch_size=batch_size)
    results_i = model_evaluate_sim(model_i, snr_range=list(range(-20, 21, 4)), num_samples=num_samples)
    results.append(results_i)
  print('MC-CV end!')
  return list(filter_snr), np.array(results)

In [ ]:
def main():
  snr_range, results = MC_CV_sim(filter_snr= list(range(0, 21, 2)),
                                num_samples= 40, num_iterations=15,
                                num_epochs= 10, batch_size= 8)
  np.save(os.path.join(PATH_RESULTS, 'results_007.npy'), results)

if __name__ == "__main__":
  main()